In [2]:
import pandas as pd
import numpy as np
import mglearn
import matplotlib.pyplot as plt
%matplotlib inline

# 評価基準の変更
scoreで評価基準をaccurancyからAUCなどに変えたい場合には、scoring引数に対応する文字列を与える。

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.svm import SVC
digits = load_digits()
y = digits.target == 9

X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=0) 

In [15]:
from sklearn.model_selection import cross_val_score
print('Default socirng : {}'.format(cross_val_score(SVC(), digits.data, digits.target == 9)))
#scoring='accurancy'としても変わらない
print('Explicit accuracy socirng : {}'.format(cross_val_score(SVC(), digits.data, digits.target == 9, scoring='accuracy')))

print('AUC socirng : {}'.format(cross_val_score(SVC(), digits.data, digits.target == 9, scoring='roc_auc')))

Default socirng : [0.89983306 0.89983306 0.89983306]
Explicit accuracy socirng : [0.89983306 0.89983306 0.89983306]
AUC socirng : [0.99372294 0.98957947 0.99594929]


GridSearchCVで最も良いパラメーターを探索する時の指標も変更することができる

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target==9, random_state=0)

param_grid = {'gamma': [0.0001,0.01,0.1,1,10]}

#デフォルトのスコア法である精度で評価
grid = GridSearchCV(SVC(), param_grid=param_grid)
grid.fit(X_train, y_train)
print('Grid Search with accuracy')
print('Best parameters;', grid.best_params_)
print('Best cross-validation score (accuracy): {:.3f}'.format(grid.best_score_))
print('Test set AUC: {:.3f}'.format(roc_auc_score(y_test, grid.decision_function(X_test))))
print('Test set accuracy: {:.3f}'.format(grid.score(X_test, y_test)))

Grid Search with accuracy
Best parameters; {'gamma': 0.0001}
Best cross-validation score (accuracy): 0.970
Test set AUC: 0.992
Test set accuracy: 0.973


In [21]:
#AUCで評価
grid = GridSearchCV(SVC(), param_grid=param_grid, scoring='roc_auc')
grid.fit(X_train, y_train)
print('Grid Search with accuracy')
print('Best parameters;', grid.best_params_)
print('Best cross-validation score (AUC): {:.3f}'.format(grid.best_score_))
print('Test set AUC: {:.3f}'.format(roc_auc_score(y_test, grid.decision_function(X_test))))
print('Test set accuracy: {:.3f}'.format(grid.score(X_test, y_test)))

Grid Search with accuracy
Best parameters; {'gamma': 0.01}
Best cross-validation score (AUC): 0.997
Test set AUC: 1.000
Test set accuracy: 1.000


評価指標に精度を用いるとgamma=0.0001が選択されるが、AUCを選択するとgamma=0.01が選択される。どちらも交差検証精度はテストセット精度と同じくらいである。しかしAUCスコアを用いて見つけたパラメーターの方が精度でもAUCでも性能が良い。これは偏ったデータセットにおいては精度は良い評価基準ではないことによるものである。  
クラス分類におけるscoringパラメーターの値として重要なものとしては、accuracy、roc_auc, f1, f1_macro, f1_micro, f1_weightedがある。回帰でよく用いられるのはr2、mean_squared_error、mean_absolute_errorである。